<a href="https://colab.research.google.com/github/aderdouri/EiCNAM/blob/master/Tutorials/Notebooks/mc_barrier_option_dupire_sensitivities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mathematical Description of the Financial Model

## Barrier Options under the Black-Scholes Model

A barrier option is a type of financial derivative whose payoff depends on whether the underlying asset's price reaches a specific barrier level during the option's life.


### Black-Scholes Formula for Barrier Options

The value of a barrier option can be derived by adjusting the standard Black-Scholes model with additional terms to account for the barrier conditions. The formulas for *up-and-out call* and *down-and-out put* options are given by:

$
C_{\text{up-out}} =
\begin{cases}
    0, & S \geq H \\\\
    S \Phi(d_1) - K e^{-rT} \Phi(d_2) - \left[ S \left(\frac{H}{S}\right)^{2\lambda} \Phi(x_1) - K e^{-rT} \left(\frac{H}{S}\right)^{2\lambda - 2} \Phi(x_2) \right], & S < H
\end{cases},
$

$
P_{\text{down-out}} =
\begin{cases}
    0, & S \leq H \\\\
    K e^{-rT} \Phi(-d_2) - S \Phi(-d_1) - \left[ K e^{-rT} \left(\frac{H}{S}\right)^{2\lambda - 2} \Phi(-x_2) - S \left(\frac{H}{S}\right)^{2\lambda} \Phi(-x_1) \right], & S > H
\end{cases},
$


### Parameter Definitions

$
d_1 = \frac{\ln(S / K) + (r + 0.5 \sigma^2)T}{\sigma \sqrt{T}}, \quad
d_2 = d_1 - \sigma \sqrt{T},
$

$
x_1 = \frac{\ln(S / H)}{\sigma \sqrt{T}} + \lambda \sigma \sqrt{T}, \quad
x_2 = x_1 - \sigma \sqrt{T},
$

$
\lambda = \frac{r + 0.5 \sigma^2}{\sigma^2}.
$

Here, $ S $ is the spot price, $ K $ is the strike price, $ H $ is the barrier level, $ r $ is the risk-free interest rate, $ T $ is the time to maturity, and $ \sigma $ is the local volatility. $ \Phi(\cdot) $ is the cumulative distribution function of the standard normal distribution.


## Sensitivity Analysis using Automatic Differentiation

The sensitivity of the barrier option price to the local volatility surface $ \sigma_{\text{loc}}(K, T) $ is computed using automatic differentiation. The sensitivity is defined as:

$
\frac{\partial V}{\partial \sigma_{\text{loc}}(K, T)},
$

where $ V $ is the price of the barrier option. By leveraging automatic differentiation, this derivative is computed directly in the computational graph without requiring finite difference approximations.


In [ ]:
import torch
import numpy as np
from torch.distributions.normal import Normal

In [ ]:
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
torch.set_printoptions(sci_mode=False)

In [ ]:
import torch
import numpy as np
from torch.distributions.normal import Normal

torch.manual_seed(42)
np.random.seed(42)

torch.set_printoptions(sci_mode=False)

In [ ]:
# Barrier option pricing with the Black-Scholes formula
def barrier_option_price(S, K, T, r, sigma, H, option_type="call", barrier_type="up-and-out"):
    # S: Spot price (scalar or tensor)
    # K: Strike price (tensor)
    # T: Time to maturity (tensor)
    # r: Risk-free rate (scalar)
    # sigma: Local volatility (tensor, same shape as K and T)
    # H: Barrier level (scalar)
    # option_type: "call" or "put"
    # barrier_type: "up-and-out" or "down-and-out"

    # Avoid division by zero for very small T
    T = torch.clamp(T, min=1e-6)

    # Standard normal distribution
    normal = Normal(0, 1)

    # Parameters for Black-Scholes formula
    d1 = (torch.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * torch.sqrt(T))
    d2 = d1 - sigma * torch.sqrt(T)

    # Parameters for the barrier option adjustment
    lambda_ = (r + 0.5 * sigma**2) / (sigma**2)
    x1 = torch.log(S / H) / (sigma * torch.sqrt(T)) + lambda_ * sigma * torch.sqrt(T)
    x2 = x1 - sigma * torch.sqrt(T)

    # Barrier option pricing logic
    if option_type == "call" and barrier_type == "up-and-out":
        # Up-and-out call
        return torch.where(S >= H, torch.zeros_like(K),  # Knocked out if spot >= barrier
                           S * normal.cdf(d1) - K * torch.exp(-r * T) * normal.cdf(d2) - \
                           (S * (H / S)**(2 * lambda_) * normal.cdf(x1) - K * torch.exp(-r * T) * (H / S)**(2 * lambda_ - 2) * normal.cdf(x2)))
    elif option_type == "put" and barrier_type == "down-and-out":
        # Down-and-out put
        return torch.where(S <= H, torch.zeros_like(K),  # Knocked out if spot <= barrier
                           K * torch.exp(-r * T) * normal.cdf(-d2) - S * normal.cdf(-d1) - \
                           (K * torch.exp(-r * T) * (H / S)**(2 * lambda_ - 2) * normal.cdf(-x2) - S * (H / S)**(2 * lambda_) * normal.cdf(-x1)))
    else:
        raise ValueError("Unsupported option or barrier type")

# Function to calculate sensitivities for barrier options
def calculate_barrier_sensitivities(S, K, T, r, sigma_surface, H, option_type="call", barrier_type="up-and-out", epsilon=1e-4):
    # S: Spot price (scalar)
    # K: Strike price (tensor)
    # T: Time to maturity (tensor)
    # r: Risk-free rate (scalar)
    # sigma_surface: Local volatility surface (tensor of shape [num_strikes, num_times])
    # H: Barrier level (scalar)

    # Expand S, K, and T to match the shape of sigma_surface
    S_exp = S * torch.ones_like(sigma_surface)
    K_exp = K.unsqueeze(1).expand_as(sigma_surface)
    T_exp = T.unsqueeze(0).expand_as(sigma_surface)

    # Calculate the option price for the base volatility surface
    base_prices = barrier_option_price(S_exp, K_exp, T_exp, r, sigma_surface, H, option_type, barrier_type)
    print('base_prices', base_prices)

    # Sensitivities with respect to volatility surface
    sensitivity_sigma = torch.zeros_like(sigma_surface)

    # Loop over the volatility surface and calculate the derivatives (sensitivities)
    for i in range(sigma_surface.shape[0]):  # Loop over strikes
        for j in range(sigma_surface.shape[1]):  # Loop over times
            # Perturb the volatility surface slightly
            sigma_perturbed = sigma_surface.clone()
            sigma_perturbed[i, j] += epsilon

            # Calculate option price for the perturbed volatility surface
            perturbed_prices = barrier_option_price(S_exp, K_exp, T_exp, r, sigma_perturbed, H, option_type, barrier_type)

            # Compute sensitivity by finite difference
            sensitivity_sigma[i, j] = (perturbed_prices[i, j] - base_prices[i, j]) / epsilon

    return sensitivity_sigma

In [ ]:
# Example parameters
S = torch.tensor(100.0)  # Spot price
#K = torch.linspace(90, 110, 10)  # Strike prices (from 90 to 110)
K = torch.linspace(90, 180, steps=10)
T = torch.linspace(0.1, 2.0, 5)  # Time to maturities (from 0.1 to 2 years)
r = torch.tensor(0.00)  # Risk-free rate (5%)
H = torch.tensor(1500.0)  # Barrier level

# Example volatility surface (local volatility from Dupire model)
sigma_surface = torch.ones((10, 5)) * 0.2  # 10 strikes, 5 maturities

# Calculate sensitivities for barrier options
sensitivity_sigma = calculate_barrier_sensitivities(S, K, T, r, sigma_surface, H)

# Print the sensitivities
print("Sensitivity with respect to the volatility surface for barrier options:")
print(sensitivity_sigma)


In [ ]:
# Barrier option pricing with the Black-Scholes formula
def barrier_option_price(S, K, T, r, sigma, H, option_type="call", barrier_type="up-and-out"):
    # S: Spot price (scalar or tensor)
    # K: Strike price (tensor)
    # T: Time to maturity (tensor)
    # r: Risk-free rate (scalar)
    # sigma: Local volatility (tensor, same shape as K and T)
    # H: Barrier level (scalar)
    # option_type: "call" or "put"
    # barrier_type: "up-and-out" or "down-and-out"

    # Avoid division by zero for very small T
    T = torch.clamp(T, min=1e-6)

    # Standard normal distribution
    normal = Normal(0, 1)

    # Parameters for Black-Scholes formula
    d1 = (torch.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * torch.sqrt(T))
    d2 = d1 - sigma * torch.sqrt(T)

    # Parameters for the barrier option adjustment
    lambda_ = (r + 0.5 * sigma**2) / (sigma**2)
    x1 = torch.log(S / H) / (sigma * torch.sqrt(T)) + lambda_ * sigma * torch.sqrt(T)
    x2 = x1 - sigma * torch.sqrt(T)

    # Barrier option pricing logic
    if option_type == "call" and barrier_type == "up-and-out":
        return torch.where(S >= H, torch.zeros_like(K),  # Knocked out if spot >= barrier
                           S * normal.cdf(d1) - K * torch.exp(-r * T) * normal.cdf(d2) - \
                           (S * (H / S)**(2 * lambda_) * normal.cdf(x1) - K * torch.exp(-r * T) * (H / S)**(2 * lambda_ - 2) * normal.cdf(x2)))
    elif option_type == "put" and barrier_type == "down-and-out":
        return torch.where(S <= H, torch.zeros_like(K),  # Knocked out if spot <= barrier
                           K * torch.exp(-r * T) * normal.cdf(-d2) - S * normal.cdf(-d1) - \
                           (K * torch.exp(-r * T) * (H / S)**(2 * lambda_ - 2) * normal.cdf(-x2) - S * (H / S)**(2 * lambda_) * normal.cdf(-x1)))
    else:
        raise ValueError("Unsupported option or barrier type")

# Function to calculate sensitivities for barrier options using automatic differentiation
def calculate_barrier_sensitivities_autodiff(S, K, T, r, sigma_surface, H, option_type="call", barrier_type="up-and-out"):
    # S: Spot price (scalar)
    # K: Strike price (tensor)
    # T: Time to maturity (tensor)
    # r: Risk-free rate (scalar)
    # sigma_surface: Local volatility surface (tensor of shape [num_strikes, num_times])
    # H: Barrier level (scalar)

    # Expand S, K, and T to match the shape of sigma_surface
    S_exp = S * torch.ones_like(sigma_surface)
    K_exp = K.unsqueeze(1).expand_as(sigma_surface)
    T_exp = T.unsqueeze(0).expand_as(sigma_surface)

    # Enable gradient tracking for the sigma_surface
    sigma_surface = sigma_surface.clone().detach().requires_grad_(True)

    # Calculate the option price for the given volatility surface
    prices = barrier_option_price(S_exp, K_exp, T_exp, r, sigma_surface, H, option_type, barrier_type)
    print('prices', prices)

    # Perform automatic differentiation to compute sensitivities
    sensitivities = torch.autograd.grad(outputs=prices.sum(), inputs=sigma_surface, create_graph=True)[0]

    return sensitivities

In [ ]:
# Example parameters
S = torch.tensor(100.0)  # Spot price
#K = torch.linspace(90, 110, 10)  # Strike prices (from 90 to 110)
K = torch.linspace(90, 180, steps=10)
T = torch.linspace(0.1, 2.0, 5)  # Time to maturities (from 0.1 to 2 years)
r = torch.tensor(0.00)  # Risk-free rate (5%)
H = torch.tensor(1500.0)  # Barrier level

# Example volatility surface (local volatility from Dupire model)
sigma_surface = torch.ones((10, 5)) * 0.2  # 10 strikes, 5 maturities

# Calculate sensitivities for barrier options using automatic differentiation
sensitivity_sigma = calculate_barrier_sensitivities_autodiff(S, K, T, r, sigma_surface, H)

# Print the sensitivities
print("Sensitivity with respect to the volatility surface for barrier options:")
print(sensitivity_sigma)

## MONTE CARLO SIMULATION

In [ ]:
# Monte Carlo simulation for barrier option pricing
def simulate_paths_mc(S0, r, sigma_surface, T, H, N_t, N_paths, strikes, times):
    dt = T / N_t
    #dt_tensor = torch.tensor(dt, dtype=torch.float32)  # Convert to PyTorch tensor
    time_grid = torch.linspace(0, T, steps=N_t + 1)
    paths = torch.zeros((N_paths, N_t + 1), dtype=torch.float32, requires_grad=False)
    paths[:, 0] = S0

    for t_idx in range(1, N_t + 1):
        t = time_grid[t_idx]
        Z = torch.randn(N_paths)  # Standard normal random variables
        S_prev = paths[:, t_idx - 1].clone()  # Clone to avoid in-place modifications

        # Interpolate local volatility for current spot and time
        sigma_t = interpolate_volatility(S_prev, t, strikes, times, sigma_surface)

        # Simulate next step of the path
        dS = r * S_prev * dt + sigma_t * S_prev * torch.sqrt(dt) * Z
        paths[:, t_idx] = S_prev + dS

    # Check for barrier breach
    breached = (paths > H).any(dim=1)  # True if the path crosses the barrier
    return paths, breached


def interpolate_volatility(S, t, strikes, times, sigma_surface):
    """
    Interpolate local volatility from the surface using bilinear interpolation.

    Args:
    - S: Spot prices (1D tensor).
    - t: Current time (scalar).
    - strikes: Tensor of strike levels.
    - times: Tensor of time levels.
    - sigma_surface: Local volatility surface (tensor, shape [num_strikes, num_times]).

    Returns:
    - Interpolated volatility values (1D tensor).
    """
    # Ensure inputs are tensors
    S = S.clone().detach()  # Ensure no in-place modifications
    t = t.clone().detach()

    # Find indices for strikes and times
    strike_idx = torch.searchsorted(strikes, S).clamp(1, len(strikes) - 1)
    time_idx = torch.searchsorted(times, t).clamp(1, len(times) - 1)

    # Get bounding indices
    strike_idx0 = strike_idx - 1
    strike_idx1 = strike_idx
    time_idx0 = time_idx - 1
    time_idx1 = time_idx

    # Get bounding values
    S0, S1 = strikes[strike_idx0], strikes[strike_idx1]
    t0, t1 = times[time_idx0], times[time_idx1]

    # Get corresponding volatilities
    vol00 = sigma_surface[strike_idx0, time_idx0]
    vol01 = sigma_surface[strike_idx0, time_idx1]
    vol10 = sigma_surface[strike_idx1, time_idx0]
    vol11 = sigma_surface[strike_idx1, time_idx1]

    # Bilinear interpolation
    vol_t0 = vol00 + (vol10 - vol00) * (S - S0) / (S1 - S0)
    vol_t1 = vol01 + (vol11 - vol01) * (S - S0) / (S1 - S0)
    vol = vol_t0 + (vol_t1 - vol_t0) * (t - t0) / (t1 - t0)

    return vol


def monte_carlo_barrier_with_sensitivities(S0, K, H, T, r, sigma_surface, N_t, N_paths, strikes, times, option_type="call"):
    # Enable gradient tracking for the local volatility surface
    sigma_surface = sigma_surface.clone().detach().requires_grad_(True)

    # Simulate paths
    paths, breached = simulate_paths_mc(S0, r, sigma_surface, T, H, N_t, N_paths, strikes, times)

    # Terminal prices for paths that did not breach the barrier
    terminal_prices = paths[:, -1]
    if option_type == "call":
        payoffs = torch.where(~breached, torch.maximum(terminal_prices - K, torch.tensor(0.0)), torch.tensor(0.0))
    elif option_type == "put":
        payoffs = torch.where(~breached, torch.maximum(K - terminal_prices, torch.tensor(0.0)), torch.tensor(0.0))
    else:
        raise ValueError("Unsupported option type")

    # Discount payoffs to today
    discounted_payoffs = torch.exp((-r * T).clone().detach()) * payoffs
    price = discounted_payoffs.mean()  # Monte Carlo estimate of the option price

    # Calculate gradients (sensitivities) with respect to the local volatility surface
    price.backward()  # Perform backpropagation
    sensitivities = sigma_surface.grad  # Gradient of price w.r.t. the local volatility surface

    return price.item(), sensitivities

In [ ]:
# Example parameters
S0 = torch.tensor(100.0)  # Spot price
K = torch.tensor(120.0)  # Strike price
H = torch.tensor(1500.0)  # Barrier level
T = torch.tensor(2.0)  # Time to maturity (years)
r = torch.tensor(0.0)  # Risk-free rate
N_t = torch.tensor(156)  # Number of time steps (weekly)
N_paths = 100000  # Number of Monte Carlo paths (reduced for testing)

sigma_surface = torch.ones((10, 5), requires_grad=True) * 0.2  # 10 strikes, 5 maturities

# Strike levels and time levels corresponding to the sigma_surface
#strikes = torch.linspace(90, 150, sigma_surface.size(0))
strikes = torch.linspace(90, 180, steps=10)
#times = torch.linspace(0.0, T, sigma_surface.size(1))
times = torch.linspace(0.1, 2.0, 5)  # Time to maturities (from 0.1 to 2 years)

# Calculate price and sensitivities
price, sensitivities = monte_carlo_barrier_with_sensitivities(S0, K, H, T, r, sigma_surface,
                                                              N_t, N_paths, strikes, times, option_type="call")

print(f"The price of the up-and-out barrier call option is: {price:.4f}")
print("Sensitivities with respect to the local volatility surface:")
print(sensitivities)

## DEBUGGING MONTE CARLO

In [ ]:
import torch
import numpy as np
from torch.distributions.normal import Normal

torch.manual_seed(42)
np.random.seed(42)

torch.set_printoptions(sci_mode=False)

In [ ]:
def interpolate_volatility(S, t, strikes, times, sigma_surface):
    """
    Interpolate local volatility from the surface using bilinear interpolation.

    Args:
    - S: Spot prices (1D tensor).
    - t: Current time (scalar).
    - strikes: Tensor of strike levels.
    - times: Tensor of time levels.
    - sigma_surface: Local volatility surface (tensor, shape [num_strikes, num_times]).

    Returns:
    - Interpolated volatility values (1D tensor).
    """
    # Ensure inputs are tensors
    S = S.clone().detach()  # Ensure no in-place modifications
    t = t.clone().detach()

    # Find indices for strikes and times
    strike_idx = torch.searchsorted(strikes, S).clamp(1, len(strikes) - 1)
    time_idx = torch.searchsorted(times, t).clamp(1, len(times) - 1)

    # Get bounding indices
    strike_idx0 = strike_idx - 1
    strike_idx1 = strike_idx
    time_idx0 = time_idx - 1
    time_idx1 = time_idx

    # Get bounding values
    S0, S1 = strikes[strike_idx0], strikes[strike_idx1]
    t0, t1 = times[time_idx0], times[time_idx1]

    # Get corresponding volatilities
    vol00 = sigma_surface[strike_idx0, time_idx0]
    vol01 = sigma_surface[strike_idx0, time_idx1]
    vol10 = sigma_surface[strike_idx1, time_idx0]
    vol11 = sigma_surface[strike_idx1, time_idx1]

    # Bilinear interpolation
    vol_t0 = vol00 + (vol10 - vol00) * (S - S0) / (S1 - S0)
    vol_t1 = vol01 + (vol11 - vol01) * (S - S0) / (S1 - S0)
    vol = vol_t0 + (vol_t1 - vol_t0) * (t - t0) / (t1 - t0)

    return vol

In [ ]:
# Monte Carlo simulation for barrier option pricing with sensitivities
def simulate_paths_mc(S0, r, sigma_surface, T, H, N_t, N_paths, strikes, times, epsilon=1e-6):
    #dt = torch.tensor(T / N_t, dtype=torch.float32)  # Convert to a PyTorch tensor

    dt = (T / N_t).clone().detach().float()  # Ensure it's a tensor without gradient


    time_grid = torch.linspace(0, T, steps=N_t + 1)
    paths = torch.zeros((N_paths, N_t + 1), dtype=torch.float32, requires_grad=False)
    paths[:, 0] = S0

    alive = torch.ones(N_paths, dtype=torch.float32)  # Track alive paths

    for t_idx in range(1, N_t + 1):
        t = time_grid[t_idx]
        Z = torch.randn(N_paths)  # Standard normal random variables
        S_prev = paths[:, t_idx - 1].clone()  # Clone to avoid in-place modifications

        # Interpolate local volatility for current spot and time
        sigma_t = interpolate_volatility(S_prev, t, strikes, times, sigma_surface)

        # Simulate next step of the path
        dS = r * S_prev * dt + sigma_t * S_prev * torch.sqrt(dt) * Z
        paths[:, t_idx] = S_prev + dS

        # Monitor barrier
        breach_high = paths[:, t_idx] > (H + epsilon)
        breach_low = paths[:, t_idx] < (H - epsilon)
        interpolate_zone = ~(breach_high | breach_low)

        # Update alive status
        alive[breach_high] = 0.0  # Definitely dead
        alive[interpolate_zone] *= 1.0 - (paths[:, t_idx][interpolate_zone] - (H - epsilon)) / (2 * epsilon)

    breached = alive < 1.0  # Paths that breached the barrier
    return paths, breached

In [ ]:
def monte_carlo_barrier_with_sensitivities(S0, K, H, T, r, sigma_surface, N_t, N_paths, strikes, times, option_type="call", epsilon=1e-6):
    # Ensure r and T are tensors
    r = torch.tensor(r, dtype=torch.float32)
    T = torch.tensor(T, dtype=torch.float32)

    # Enable gradient tracking for the local volatility surface
    sigma_surface = sigma_surface.clone().detach().requires_grad_(True)

    # Simulate paths
    paths, breached = simulate_paths_mc(S0, r, sigma_surface, T, H, N_t, N_paths, strikes, times, epsilon)

    # Terminal prices for paths that did not breach the barrier
    terminal_prices = paths[:, -1]
    if option_type == "call":
        payoffs = torch.where(~breached, torch.maximum(terminal_prices - K, torch.tensor(0.0)), torch.tensor(0.0))
    elif option_type == "put":
        payoffs = torch.where(~breached, torch.maximum(K - terminal_prices, torch.tensor(0.0)), torch.tensor(0.0))
    else:
        raise ValueError("Unsupported option type")

    # Discount payoffs to today
    discounted_payoffs = torch.exp(-r * T) * payoffs
    price = discounted_payoffs.mean()  # Monte Carlo estimate of the option price

    # Calculate gradients (sensitivities) with respect to the local volatility surface
    price.backward()  # Perform backpropagation
    sensitivities = sigma_surface.grad  # Gradient of price w.r.t. the local volatility surface

    return price.item(), sensitivities

In [ ]:
# Example parameters
S0 = torch.tensor(100.0)  # Spot price
K = torch.tensor(120.0)  # Strike price
H = torch.tensor(1500.0)  # Barrier level
T = torch.tensor(2.0)  # Time to maturity (years)
r = torch.tensor(0.0)  # Risk-free rate
N_t = torch.tensor(156)  # Number of time steps (weekly)
N_paths = 100000  # Number of Monte Carlo paths (reduced for testing)

# Local volatility surface
sigma_surface = torch.ones((10, 5), requires_grad=True) * 0.2  # 10 strikes, 5 maturities

# Strike levels and time levels corresponding to the sigma_surface
strikes = torch.linspace(90, 180, steps=10)  # Strike levels
times = torch.linspace(0.1, 2.0, 5)  # Time to maturities (from 0.1 to 2 years)

# Run the Monte Carlo simulation with sensitivities
price, sensitivities = monte_carlo_barrier_with_sensitivities(
    S0=S0.item(),
    K=K.item(),
    H=H.item(),
    T=T.item(),
    r=r.item(),
    sigma_surface=sigma_surface,
    N_t=N_t.item(),
    N_paths=N_paths,
    strikes=strikes,
    times=times,
    option_type="call",
    epsilon=0.05
)

# Print results
print(f"Barrier Option Price: {price:.4f}")
print("Sensitivities with respect to the local volatility surface:")
print(sensitivities)

## Automatic Differentiation check

In [ ]:
# Barrier option pricing with the Black-Scholes formula (with epsilon logic)
def barrier_option_price(S, K, T, r, sigma, H, option_type="call", barrier_type="up-and-out", epsilon=1e-4):
    # S: Spot price (scalar or tensor)
    # K: Strike price (tensor)
    # T: Time to maturity (tensor)
    # r: Risk-free rate (scalar)
    # sigma: Local volatility (tensor, same shape as K and T)
    # H: Barrier level (scalar)
    # epsilon: Threshold for the probabilistic survival region
    # option_type: "call" or "put"
    # barrier_type: "up-and-out" or "down-and-out"

    # Avoid division by zero for very small T
    T = torch.clamp(T, min=1e-6)

    # Standard normal distribution
    normal = Normal(0, 1)

    # Parameters for Black-Scholes formula
    d1 = (torch.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * torch.sqrt(T))
    d2 = d1 - sigma * torch.sqrt(T)

    # Parameters for the barrier option adjustment
    lambda_ = (r + 0.5 * sigma**2) / (sigma**2)
    x1 = torch.log(S / H) / (sigma * torch.sqrt(T)) + lambda_ * sigma * torch.sqrt(T)
    x2 = x1 - sigma * torch.sqrt(T)

    # Barrier option pricing logic with epsilon logic
    if option_type == "call" and barrier_type == "up-and-out":
        # Check the relationship of S to H with epsilon
        survival_probability = torch.where(
            S > H + epsilon,
            torch.zeros_like(S),  # Definitely knocked out
            torch.where(
                S < H - epsilon,
                torch.ones_like(S),  # Definitely alive
                1.0 - (S - H + epsilon) / (2 * epsilon)  # Probabilistic survival
            )
        )
        # Adjust price for probabilistic survival
        price = survival_probability * (
            S * normal.cdf(d1) - K * torch.exp(-r * T) * normal.cdf(d2) -
            (S * (H / S)**(2 * lambda_) * normal.cdf(x1) -
             K * torch.exp(-r * T) * (H / S)**(2 * lambda_ - 2) * normal.cdf(x2))
        )
        return price

    elif option_type == "put" and barrier_type == "down-and-out":
        # Check the relationship of S to H with epsilon
        survival_probability = torch.where(
            S < H - epsilon,
            torch.zeros_like(S),  # Definitely knocked out
            torch.where(
                S > H + epsilon,
                torch.ones_like(S),  # Definitely alive
                1.0 - (H - S + epsilon) / (2 * epsilon)  # Probabilistic survival
            )
        )
        # Adjust price for probabilistic survival
        price = survival_probability * (
            K * torch.exp(-r * T) * normal.cdf(-d2) - S * normal.cdf(-d1) -
            (K * torch.exp(-r * T) * (H / S)**(2 * lambda_ - 2) * normal.cdf(-x2) -
             S * (H / S)**(2 * lambda_) * normal.cdf(-x1))
        )
        return price

    else:
        raise ValueError("Unsupported option or barrier type")

# Function to calculate sensitivities for barrier options using automatic differentiation
def calculate_barrier_sensitivities_autodiff(S, K, T, r, sigma_surface, H, option_type="call", barrier_type="up-and-out", epsilon=1e-4):
    # S: Spot price (scalar)
    # K: Strike price (tensor)
    # T: Time to maturity (tensor)
    # r: Risk-free rate (scalar)
    # sigma_surface: Local volatility surface (tensor of shape [num_strikes, num_times])
    # H: Barrier level (scalar)
    # epsilon: Threshold for the probabilistic survival region

    # Expand S, K, and T to match the shape of sigma_surface
    S_exp = S * torch.ones_like(sigma_surface)
    K_exp = K.unsqueeze(1).expand_as(sigma_surface)
    T_exp = T.unsqueeze(0).expand_as(sigma_surface)

    # Enable gradient tracking for the sigma_surface
    sigma_surface = sigma_surface.clone().detach().requires_grad_(True)

    # Calculate the option price for the given volatility surface
    prices = barrier_option_price(S_exp, K_exp, T_exp, r, sigma_surface, H, option_type, barrier_type, epsilon)

    # Perform automatic differentiation to compute sensitivities
    sensitivities = torch.autograd.grad(outputs=prices.sum(), inputs=sigma_surface, create_graph=True)[0]

    return sensitivities


In [ ]:
# Example parameters
S = torch.tensor(100.0)  # Spot price
#K = torch.linspace(90, 110, 10)  # Strike prices (from 90 to 110)
K = torch.linspace(90, 180, steps=10)
T = torch.linspace(0.1, 2.0, 5)  # Time to maturities (from 0.1 to 2 years)
r = torch.tensor(0.00)  # Risk-free rate (5%)
H = torch.tensor(1500.0)  # Barrier level

# Example volatility surface (local volatility from Dupire model)
sigma_surface = torch.ones((10, 5)) * 0.2  # 10 strikes, 5 maturities

# Calculate sensitivities for barrier options using automatic differentiation
sensitivity_sigma = calculate_barrier_sensitivities_autodiff(S, K, T, r, sigma_surface, H)

# Print the sensitivities
print("Sensitivity with respect to the volatility surface for barrier options:")
print(sensitivity_sigma)